In [1]:
!pip install -U vnstock

In [2]:
pip install ta

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install keras-self-attention

In [5]:
!pip install keras-tuner

In [6]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Phiên bản Vnai 2.1.9 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnai --upgrade`.
Lịch sử phiên bản: https://pypi.org/project/vnai/#history
Phiên bản hiện tại 2.0.4

In [7]:
from vnstock import *
import pandas as pd
import os

In [8]:
from collections import Counter

In [15]:
stock = Vnstock().stock(symbol='TCB', source='VCI')
tcb_df = stock.quote.history(start='2019-01-01', end='2024-12-31')
tcb_df['ticker'] = 'TCB'
tcb_df

,time,open,high,low,close,volume,ticker
0,2019-01-02,12.61,12.63,12.41,12.41,1329030,TCB
1,2019-01-03,12.46,12.49,12.12,12.12,1811430,TCB
2,2019-01-04,12.12,12.12,11.27,12.10,2970830,TCB
3,2019-01-07,12.20,12.36,12.17,12.17,1436930,TCB
4,2019-01-08,12.20,12.22,12.15,12.17,843170,TCB
...,...,...,...,...,...,...,...
1495,2024-12-25,24.00,24.90,23.95,24.45,18370600,TCB
1496,2024-12-26,24.50,24.55,24.30,24.35,5475600,TCB
1497,2024-12-27,24.50,24.90,24.45,24.60,15395600,TCB
1498,2024-12-30,24.75,24.75,24.35,24.40,10145800,TCB


In [16]:
# stock = Vnstock().stock(symbol='VNINDEX', source='VCI')
# stock.quote.history(start='2019-01-01', end='2024-12-31')
finance_data_tcb = stock.finance.ratio(period='quarter', lang='vi', dropna=True)
display(finance_data_tcb)

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Meta          Chỉ tiêu cơ cấu nguồn vốn                                     \
     CP   Năm Kỳ                   Nợ/VCSH TSCĐ / Vốn CSH Vốn CSH/Vốn điều lệ   
0   TCB  2025  2                  5.412057       0.075345            2.290596   
1   TCB  2025  1                  5.425455       0.078723            2.179135   
2   TCB  2024  4                  5.616203       0.084270            2.094023   
3   TCB  2024  3                  5.421461       0.088332            2.043468   
4   TCB  2024  2                  5.555726       0.069482            1.961140   
5   TCB  2024  1                  5.423703       0.067154            1.951530   
6   TCB  2023  4                  5.454243       0.067559            1.863140   
7   TCB  2023  3                  5.145520       0.067432            1.799469   
8   TCB  2023  2                  4.981059       0.071387            1.733440   
9   TCB  2023  1                  5.133304       0.070147            1.669751   
10  TCB  2022  4                  5.162951       0.074158            1.605483   
11  TCB  2022  3                  5.108815       0.066260            1.555576   
12  TCB  2022  2                  4.970209       0.069111            1.478802   
13  TCB  2022  1                  5.240951       0.073694            1.395443   
14  TCB  2021  4                  5.112639       0.077636            1.317166   
15  TCB  2021  3                  5.128175       0.080399            1.251045   
16  TCB  2021  2                  5.010700       0.080318            1.187583   
17  TCB  2021  1                  4.851644       0.059757            1.119525   
18  TCB  2020  4                  4.891633       0.061830            1.056141   
19  TCB  2020  3                  4.685754       0.060510            0.999431   
20  TCB  2020  2                  4.870766       0.061190            0.954432   
21  TCB  2020  1                  5.066004       0.054079            0.914255   
22  TCB  2019  4                  5.181446       0.051678            0.878614   
23  TCB  2019  3                  5.235730       0.059457            0.834281   
24  TCB  2019  2                  5.398603       0.061012            0.797833   
25  TCB  2019  1                  5.053123       0.057620            0.762577   
26  TCB  2018  4                  5.198767       0.033189            0.732962   
27  TCB  2018  3                  5.301320       0.032054            0.700383   
28  TCB  2018  2                  0.000000       0.031389            0.671344   
29  TCB  2018  1                  6.261648       0.039419            0.532436   
30  TCB  2017  4                  9.003154       0.056123            0.381193   
31  TCB  2017  3                 11.015619       0.075199            0.275069   
32  TCB  2017  2                  9.681854       0.067731            0.307878   
33  TCB  2017  1                 10.258785       0.074018            0.292328   
34  TCB  2016  4                 11.016615       0.080620            0.277881   
35  TCB  2016  3                 10.885701       0.080739            0.265295   
36  TCB  2016  2                  0.000000       0.088085            0.251040   
37  TCB  2016  1                 10.736066       0.051902            0.240013   
38  TCB  2015  4                  0.000000       0.053369            0.233948   
39  TCB  2015  3                  0.000000       0.057603            0.228746   
40  TCB  2015  2                  0.000000       0.061913            0.223006   
41  TCB  2015  1                  0.000000       0.066120            0.216485   
42  TCB  2014  4                 10.737702       0.069102            0.212315   
43  TCB  2014  3                 10.521067       0.067195            0.209803   
44  TCB  2014  2                 10.669270       0.069721            0.207518   
45  TCB  2014  1                 10.359780       0.067874            0.204516   
46  TCB  2013  4                 10.414934       0.074191            0.197033   
47  TCB  2013  3                 1

In [ ]:
stock = Vnstock().stock(symbol='VNINDEX', source='VCI')
stock.quote.history(start='2020-01-01', end='2025-05-25')

2025-04-23 13:39:34 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.
INFO:vnstock.common.data.data_explorer:Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


,time,open,high,low,close,volume
0,2020-01-02,960.26,966.67,959.67,966.67,131523160
1,2020-01-03,968.72,970.88,965.14,965.14,149202550
2,2020-01-06,962.79,963.20,955.59,955.79,140978550
3,2020-01-07,955.39,959.46,953.19,958.88,130272030
4,2020-01-08,954.13,955.84,945.28,948.98,179374340
...,...,...,...,...,...,...
1319,2025-04-17,1210.30,1217.25,1197.50,1217.25,756538553
1320,2025-04-18,1228.37,1234.67,1219.12,1219.12,1064573281
1321,2025-04-21,1219.38,1222.77,1201.63,1207.07,880575750
1322,2025-04-22,1207.07,1207.07,1136.98,1197.13,1699625685


In [9]:

def get_previous_and_last_year_quarter(input_date):
    # Lấy tháng và năm từ input_date
    month = input_date.month
    year = input_date.year

    # Xác định quý hiện tại dựa vào tháng
    if 1 <= month <= 3:
        current_quarter = 1
    elif 4 <= month <= 6:
        current_quarter = 2
    elif 7 <= month <= 9:
        current_quarter = 3
    else:
        current_quarter = 4

    # Tìm quý trước đó
    previous_quarter = current_quarter - 1
    previous_year = year

    # Nếu quý trước là 0, thì sẽ chuyển về quý 4 của năm trước
    if previous_quarter == 0:
        previous_quarter = 4
        previous_year -= 1

    # Xác định quý cùng kỳ năm trước
    last_year = previous_year - 1

    # Format kết quả
    previous_quarter_str = f'Q{previous_quarter}-{previous_year}'
    last_year_quarter_str = f'Q{previous_quarter}-{last_year}'

    # return [previous_quarter_str, last_year_quarter_str]
    return previous_quarter, previous_year, last_year

In [10]:
import datetime
from dateutil.relativedelta import relativedelta

In [11]:
# Ví dụ sử dụng
previous_quarter, previous_year, last_year = get_previous_and_last_year_quarter(datetime.date(2024, 1, 10)) #This was changed from datetime.date to datetime.date
print(previous_quarter, previous_year, last_year)

4 2023 2022


In [12]:
def get_last_n_quarters(input_date, n=8):
    """
    Trả về danh sách [(quý, năm)] của n quý GẦN NHẤT, không bao gồm quý hiện tại.
    """
    quarters = []
    current_date = pd.to_datetime(input_date)

    # Bắt đầu từ quý TRƯỚC của ngày input
    current_date -= relativedelta(months=3)

    for _ in range(n):
        quarter = (current_date.month - 1) // 3 + 1
        year = current_date.year
        quarters.append((quarter, year))
        current_date -= relativedelta(months=3)

    return quarters

In [13]:
quarter = get_last_n_quarters(datetime.date(2024, 10, 10), n=8)

In [14]:
quarter

[(3, 2024),
 (2, 2024),
 (1, 2024),
 (4, 2023),
 (3, 2023),
 (2, 2023),
 (1, 2023),
 (4, 2022)]

In [15]:
def get_latest_quarter_end_date(input_date):
    """
    Trả về ngày kết thúc của quý gần nhất TRƯỚC input_date
    """
    input_date = pd.to_datetime(input_date)
    year = input_date.year
    month = input_date.month

    if month <= 3:
        return pd.Timestamp(year=year-1, month=12, day=31)
    elif month <= 6:
        return pd.Timestamp(year=year, month=3, day=31)
    elif month <= 9:
        return pd.Timestamp(year=year, month=6, day=30)
    else:
        return pd.Timestamp(year=year, month=9, day=30)

In [16]:
a = get_latest_quarter_end_date(datetime.date(2024, 1, 10))

In [17]:
a

Timestamp('2023-12-31 00:00:00')

In [28]:
# Thay thế function cũ bằng version mới
def crawl_finance_data_vn30(symbols, save_folder='finance_data_vn30'):
    """
    Crawl dữ liệu tài chính cho VN30 - Fixed MultiIndex issue
    """
    os.makedirs(save_folder, exist_ok=True)
    
    for symbol in symbols:
        try:
            print(f"🔄 Processing: {symbol}...")
            
            stock = Vnstock().stock(symbol=symbol, source='VCI')
            finance_data = stock.finance.ratio(period='quarter', lang='vi', dropna=True)
            finance_data
            
            if not finance_data.empty:
                # Fix MultiIndex columns
                finance_data.columns = ['_'.join(col) if isinstance(col, tuple) else str(col) 
                                      for col in finance_data.columns]
                
                # Reset index và thêm symbol
                df_final = finance_data.reset_index()
                df_final['Symbol'] = symbol
                
                # Lưu file
                file_path = os.path.join(save_folder, f"{symbol}.xlsx")
                df_final.to_excel(file_path, index=False)
                print(f"✅ Saved {symbol}: {len(df_final)} rows")
                
            else:
                print(f"⚠️ No data for {symbol}")
                
        except Exception as e:
            print(f"❌ Error {symbol}: {e}")
            continue
    
    print("🎉 Completed!")

In [29]:
# Danh sách 30 cổ phiếu VN30
vn30_symbols = [
    'ACB', 'BCM', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB', 'HPG',
    'LPB', 'MBB', 'MSN', 'MWG', 'PLX', 'SAB', 'SHB', 'SSB', 'SSI', 'STB',
    'TCB', 'TPB', 'VCB', 'VHM', 'VIB', 'VIC', 'VJC', 'VNM', 'VPB', 'VRE'
]


crawl_finance_data_vn30(
    symbols=vn30_symbols,
    save_folder=r'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\TA_FA\data\v2\finance_data'
)

🔄 Processing: ACB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Saved ACB: 50 rows
🔄 Processing: BCM...
✅ Saved BCM: 29 rows
🔄 Processing: BID...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Saved BID: 49 rows
🔄 Processing: BVH...


...

✅ Saved VCB: 49 rows
🔄 Processing: VHM...
✅ Saved VHM: 35 rows
🔄 Processing: VIB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Saved VIB: 49 rows
🔄 Processing: VIC...
✅ Saved VIC: 50 rows
🔄 Processing: VJC...
✅ Saved VJC: 34 rows
🔄 Processing: VNM...
✅ Saved VNM: 49 rows
🔄 Processing: VPB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Saved VPB: 49 rows
🔄 Processing: VRE...
✅ Saved VRE: 32 rows
🎉 Completed!


In [32]:
from IPython.display import display
for symbol in vn30_symbols:
    try:
        stock = Vnstock().stock(symbol=symbol, source='VCI')
        finance_data = stock.finance.ratio(period='quarter', lang='vi', dropna=True)
        print(f"=================================================={symbol}=============================================")
        display(finance_data)
        print(f"=====================================================================================================")
    except Exception as e:
        print(f"Error processing {symbol}: {e}")
        continue

==================================================ACB=============================================


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Meta          Chỉ tiêu cơ cấu nguồn vốn                                     \
     CP   Năm Kỳ                   Nợ/VCSH TSCĐ / Vốn CSH Vốn CSH/Vốn điều lệ   
0   ACB  2025  1                  9.240355       0.061005            1.695161   
1   ACB  2024  4                  9.352125       0.064849            1.624825   
2   ACB  2024  3                  8.858604       0.058881            1.535128   
3   ACB  2024  2                  9.290830       0.062380            1.456058   
4   ACB  2024  1                  8.724440       0.062450            1.456020   
5   ACB  2023  4                  9.130151       0.067120            1.381365   
6   ACB  2023  3                  8.704559       0.066554            1.300949   
7   ACB  2023  2                  9.035334       0.068278            1.223893   
8   ACB  2023  1                  8.774859       0.066006            1.217332   
9   ACB  2022  4                  9.401935       0.068129            1.137679   
10  ACB  2022  3                  9.067578       0.066993            1.085039   
11  ACB  2022  2                  9.443214       0.069625            1.013617   
12  ACB  2022  1                  9.966802       0.076310            0.938418   
13  ACB  2021  4                 10.754104       0.083797            0.874127   
14  ACB  2021  3                 10.282250       0.085829            0.827064   
15  ACB  2021  2                 10.670941       0.091710            0.786119   
16  ACB  2021  1                 10.851374       0.098095            0.738405   
17  ACB  2020  4                 11.540286       0.106712            0.690102   
18  ACB  2020  3                 11.720578       0.113291            0.640864   
19  ACB  2020  2                 11.868276       0.121825            0.600243   
20  ACB  2020  1                 12.171197       0.128266            0.572597   
21  ACB  2019  4                 12.812695       0.135851            0.540283   
22  ACB  2019  3                 13.121225       0.137624            0.493790   
23  ACB  2019  2                 13.724449       0.139122            0.463993   
24  ACB  2019  1                 14.002276       0.143898            0.435759   
25  ACB  2018  4                 14.669180       0.153840            0.409175   
26  ACB  2018  3                 15.024454       0.160896            0.379990   
27  ACB  2018  2                  0.000000       0.170651            0.356630   
28  ACB  2018  1                 16.456520       0.176367            0.334405   
29  ACB  2017  4                 16.735565       0.187614            0.312087   
30  ACB  2017  3                 16.523023       0.196855            0.300261   
31  ACB  2017  2                 16.860526       0.204633            0.289767   
32  ACB  2017  1                 16.280560       0.199433            0.283376   
33  ACB  2016  4                 15.617052       0.202703            0.273772   
34  ACB  2016  3                 15.674916       0.200418            0.267201   
35  ACB  2016  2                 15.567581       0.188499            0.260659   
36  ACB  2016  1                 15.016311       0.189328            0.255096   
37  ACB  2015  4                  0.000000       0.193905            0.248947   
38  ACB  2015  3                  0.000000       0.196589            0.245588   
39  ACB  2015  2                 14.353217       0.206608            0.240282   
40  ACB  2015  1                  0.000000       0.201880            0.246851   
41  ACB  2014  4                 13.487810       0.226223            0.241350   
42  ACB  2014  3                 13.342019       0.211001            0.243493   
43  ACB  2014  2                 13.446861       0.217349            0.238914   
44  ACB  2014  1                 12.586751       0.196432            0.248444   
45  ACB  2013  4                 12.323440       0.204186            0.243391   
46  ACB  2013  3                 11.526566       0.133516            0.249403   
47  ACB  2013  2                 1

==================================================BCM=============================================


Meta          Chỉ tiêu cơ cấu nguồn vốn                           \
     CP   Năm Kỳ          (Vay NH+DH)/VCSH   Nợ/VCSH TSCĐ / Vốn CSH   
0   BCM  2025  1                  1.836911  1.817847       0.048848   
1   BCM  2024  4                  1.876853  1.842506       0.051364   
2   BCM  2024  3                  1.649241  1.712412       0.053604   
3   BCM  2024  2                  1.682973  1.776227       0.056578   
4   BCM  2024  1                  1.658192  1.769024       0.057672   
5   BCM  2023  4                  1.543052  1.743501       0.061542   
6   BCM  2023  3                  1.582835  1.751564       0.066673   
7   BCM  2023  2                  1.421240  1.734822       0.069712   
8   BCM  2023  1                  1.521932  1.705092       0.069091   
9   BCM  2022  4                  1.494672  1.690957       0.069756   
10  BCM  2022  3                  1.399997  1.647860       0.065664   
11  BCM  2022  2                  1.460085  1.712787       0.067429   
12  BCM  2022  1                  1.605246  1.823946       0.068819   
13  BCM  2021  4                  1.692685  1.855215       0.072741   
14  BCM  2021  3                  1.750101  1.958565       0.076061   
15  BCM  2021  2                  1.613917  1.934253       0.077652   
16  BCM  2021  1                  1.335025  1.722108       0.070782   
17  BCM  2020  4                  1.417331  1.879681       0.108694   
18  BCM  2020  3                  1.383833  1.873549       0.112873   
19  BCM  2020  2                  1.220309  1.818152       0.117540   
20  BCM  2020  1                  1.225371  1.742451       0.112929   
21  BCM  2019  4                  1.259462  1.762874       0.122912   
22  BCM  2019  3                  1.880149  2.077835       0.128397   
23  BCM  2019  2                  2.019927  2.223334       0.110293   
24  BCM  2019  1                  2.179457  2.210969       0.122825   
25  BCM  2018  4                  2.128022  2.339376       0.137331   
26  BCM  2018  3                  0.000000  0.000000       0.134007   
27  BCM  2018  2                  0.000000  0.000000       0.142351   
28  BCM  2018  1                  0.000000  0.000000       0.141398   

                       Chỉ tiêu hiệu quả hoạt động                 \
   Vốn CSH/Vốn điều lệ           Vòng quay tài sản Vòng quay TSCĐ   
0             2.033310                    0.110327       5.855874   
1             1.997990                    0.093387       4.716208   
2             1.939240                    0.152517       7.347912   
3             1.907357                    0.154217       7.066170   
4             1.886608                    0.157599       6.828617   
5             1.844348                    0.154971       6.554361   
6             1.736533                    0.079719       3.160649   
7             1.719920                    0.103114       4.064887   
8             1.735474                    0.119533       4.742191   
9             1.723417                    0.133818       5.280988   
10            1.796293                    0.177250       7.151926   
11            1.778428                    0.147513       5.898814   
12            1.693725                    0.143726       5.656657   
13            1.620067                    0.143471       4.600556   
14            1.575817                    0.133133       4.590170   
15            1.595824                    0.150951       4.663227   
16            1.645683                    0.170047       4.704379   
17            1.616887                    0.168424       4.216326   
18            1.569283                    0.195587       4.757217   
19            1.522972                    0.194440       4.657141   
20            1.551012                    0.185118       4.646056   
21            1.450218                    0.185197       4.597302   
22            1.393326                    0.138283       3.617642   
23            1.350142                    0.146140       4.092323   
24     

...

==================================================VPB=============================================


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Meta          Chỉ tiêu cơ cấu nguồn vốn                                     \
     CP   Năm Kỳ                   Nợ/VCSH TSCĐ / Vốn CSH Vốn CSH/Vốn điều lệ   
0   VPB  2025  1                  5.573736       0.012837            1.905909   
1   VPB  2024  4                  5.272932       0.013742            1.856273   
2   VPB  2024  3                  5.038283       0.013401            1.792806   
3   VPB  2024  2                  5.260287       0.013952            1.740319   
4   VPB  2024  1                  4.753336       0.013407            1.801598   
5   VPB  2023  4                  4.848305       0.013747            1.763901   
6   VPB  2023  3                  6.093388       0.017264            1.386345   
7   VPB  2023  2                  5.877421       0.016354            1.355746   
8   VPB  2023  1                  5.447564       0.016962            1.324662   
9   VPB  2022  4                  5.096630       0.017950            1.304738   
10  VPB  2022  3                  4.821273       0.018013            1.290235   
11  VPB  2022  2                  5.173889       0.018108            1.241805   
12  VPB  2022  1                  4.923442       0.019012            1.199936   
13  VPB  2021  4                  5.344706       0.021665            1.089644   
14  VPB  2021  3                  6.692759       0.027651            0.785519   
15  VPB  2021  2                  6.527986       0.029152            0.756393   
16  VPB  2021  1                  6.790673       0.032206            0.705771   
17  VPB  2020  4                  6.937085       0.035285            0.665415   
18  VPB  2020  3                  7.323407       0.036682            0.626755   
19  VPB  2020  2                  7.414434       0.039393            0.598377   
20  VPB  2020  1                  7.831444       0.041400            0.561182   
21  VPB  2019  4                  7.936423       0.045549            0.532114   
22  VPB  2019  3                  7.847257       0.045553            0.510356   
23  VPB  2019  2                  8.127111       0.049628            0.481583   
24  VPB  2019  1                  7.902188       0.053989            0.456015   
25  VPB  2018  4                  8.303323       0.056492            0.437993   
26  VPB  2018  3                  8.260527       0.029770            0.403167   
27  VPB  2018  2                  0.000000       0.028739            0.415792   
28  VPB  2018  1                  7.989601       0.024524            0.398734   
29  VPB  2017  4                  8.353281       0.027228            0.374257   
30  VPB  2017  3                  8.189268       0.027320            0.348180   
31  VPB  2017  2                 11.739615       0.034077            0.246068   
32  VPB  2017  1                 11.305336       0.037452            0.235428   
33  VPB  2016  4                 12.318035       0.036063            0.218159   
34  VPB  2016  3                 12.036511       0.035310            0.198916   
35  VPB  2016  2                  0.000000       0.038096            0.181651   
36  VPB  2016  1                 13.322523       0.035175            0.179078   
37  VPB  2015  4                  0.000000       0.038059            0.168755   
38  VPB  2015  3                  0.000000       0.047103            0.160667   
39  VPB  2015  2                  0.000000       0.050569            0.150224   
40  VPB  2015  1                  0.000000       0.064849            0.118578   
41  VPB  2014  4                  0.000000       0.067141            0.113189   
42  VPB  2014  3                 14.694205       0.069984            0.110340   
43  VPB  2014  2                 14.694205       0.065247            0.104201   
44  VPB  2014  1                 14.694205       0.051683            0.099401   
45  VPB  2013  4                 14.694205       0.054165            0.097388   
46  VPB  2013  3                 15.826128       0.054735            0.089573   
47  VPB  2013  2                 1

==================================================VRE=============================================


Meta          Chỉ tiêu cơ cấu nguồn vốn                           \
     CP   Năm Kỳ          (Vay NH+DH)/VCSH   Nợ/VCSH TSCĐ / Vốn CSH   
0   VRE  2025  1                  0.165029  0.298069       0.004913   
1   VRE  2024  4                  0.168309  0.317335       0.005281   
2   VRE  2024  3                  0.136013  0.302115       0.005534   
3   VRE  2024  2                  0.153510  0.310486       0.005889   
4   VRE  2024  1                  0.156898  0.259762       0.010376   
5   VRE  2023  4                  0.160715  0.259786       0.010928   
6   VRE  2023  3                  0.116930  0.216190       0.011127   
7   VRE  2023  2                  0.120960  0.246920       0.011779   
8   VRE  2023  1                  0.154572  0.285027       0.012108   
9   VRE  2022  4                  0.158804  0.277531       0.012521   
10  VRE  2022  3                  0.162349  0.225601       0.012864   
11  VRE  2022  2                  0.166315  0.216033       0.013441   
12  VRE  2022  1                  0.203523  0.215998       0.013898   
13  VRE  2021  4                  0.205402  0.235643       0.014315   
14  VRE  2021  3                  0.205586  0.236411       0.014325   
15  VRE  2021  2                  0.205744  0.242883       0.014234   
16  VRE  2021  1                  0.292359  0.320857       0.014610   
17  VRE  2020  4                  0.389701  0.357263       0.015348   
18  VRE  2020  3                  0.402364  0.413166       0.015866   
19  VRE  2020  2                  0.273086  0.394835       0.016607   
20  VRE  2020  1                  0.202281  0.345040       0.017116   
21  VRE  2019  4                  0.205911  0.329648       0.017582   
22  VRE  2019  3                  0.198005  0.341170       0.007940   
23  VRE  2019  2                  0.203120  0.362716       0.008175   
24  VRE  2019  1                  0.190469  0.326633       0.010303   
25  VRE  2018  4                  0.194422  0.356880       0.010797   
26  VRE  2018  3                  0.199210  0.322217       0.010190   
27  VRE  2018  2                  0.000000  0.000000       0.011111   
28  VRE  2018  1                  0.343346  0.431167       0.006308   
29  VRE  2017  4                  0.349962  0.461376       0.006540   
30  VRE  2017  3                  0.000000  0.000000       0.007359   
31  VRE  2017  2                  0.000000  0.000000       0.007088   

                       Chỉ tiêu hiệu quả hoạt động                 \
   Vốn CSH/Vốn điều lệ           Vòng quay tài sản Vòng quay TSCĐ   
0             1.850726                    0.162748      39.431854   
1             1.800168                    0.173778      28.165477   
2             1.753564                    0.181105      28.644528   
3             1.714627                    0.214935      28.491174   
4             1.670765                    0.217765      24.586858   
5             1.624294                    0.216731      23.554699   
6             1.578455                    0.216885      22.950056   
7             1.521916                    0.191875      19.632658   
8             1.478950                    0.190464      18.747108   
9             1.433491                    0.182724      17.183921   
10            1.399743                    0.170948      15.356739   
11            1.365566                    0.141384      12.389640   
12            1.332368                    0.133124      11.562754   
13            1.316154                    0.151658      13.253291   
14            1.310920                    0.200422      17.657106   
15            1.309874                    0.222159      19.724310   
16            1.293229                    0.223995      19.691393   
17            1.259687                    0.220181      18.026693   
18            1.217839                    0.207433      16.949250   
19            1.193261                    0.222916      20.418463   
20            1.178525                    0.234814      24.934808   
2

In [21]:
start_date='2018-01-01'
end_date='2023-12-31'

In [24]:
stock = Vnstock().stock(symbol='ACB', source='VCI')
df = stock.quote.history(start=start_date, end=end_date)
finance_data = stock.finance.ratio(period='quarter', lang='vi', dropna=True)
finance_data

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Meta          Chỉ tiêu cơ cấu nguồn vốn                                     \
     CP   Năm Kỳ                   Nợ/VCSH TSCĐ / Vốn CSH Vốn CSH/Vốn điều lệ   
0   ACB  2025  1                  9.240355       0.061005            1.695161   
1   ACB  2024  4                  9.352125       0.064849            1.624825   
2   ACB  2024  3                  8.858604       0.058881            1.535128   
3   ACB  2024  2                  9.290830       0.062380            1.456058   
4   ACB  2024  1                  8.724440       0.062450            1.456020   
5   ACB  2023  4                  9.130151       0.067120            1.381365   
6   ACB  2023  3                  8.704559       0.066554            1.300949   
7   ACB  2023  2                  9.035334       0.068278            1.223893   
8   ACB  2023  1                  8.774859       0.066006            1.217332   
9   ACB  2022  4                  9.401935       0.068129            1.137679   
10  ACB  2022  3                  9.067578       0.066993            1.085039   
11  ACB  2022  2                  9.443214       0.069625            1.013617   
12  ACB  2022  1                  9.966802       0.076310            0.938418   
13  ACB  2021  4                 10.754104       0.083797            0.874127   
14  ACB  2021  3                 10.282250       0.085829            0.827064   
15  ACB  2021  2                 10.670941       0.091710            0.786119   
16  ACB  2021  1                 10.851374       0.098095            0.738405   
17  ACB  2020  4                 11.540286       0.106712            0.690102   
18  ACB  2020  3                 11.720578       0.113291            0.640864   
19  ACB  2020  2                 11.868276       0.121825            0.600243   
20  ACB  2020  1                 12.171197       0.128266            0.572597   
21  ACB  2019  4                 12.812695       0.135851            0.540283   
22  ACB  2019  3                 13.121225       0.137624            0.493790   
23  ACB  2019  2                 13.724449       0.139122            0.463993   
24  ACB  2019  1                 14.002276       0.143898            0.435759   
25  ACB  2018  4                 14.669180       0.153840            0.409175   
26  ACB  2018  3                 15.024454       0.160896            0.379990   
27  ACB  2018  2                  0.000000       0.170651            0.356630   
28  ACB  2018  1                 16.456520       0.176367            0.334405   
29  ACB  2017  4                 16.735565       0.187614            0.312087   
30  ACB  2017  3                 16.523023       0.196855            0.300261   
31  ACB  2017  2                 16.860526       0.204633            0.289767   
32  ACB  2017  1                 16.280560       0.199433            0.283376   
33  ACB  2016  4                 15.617052       0.202703            0.273772   
34  ACB  2016  3                 15.674916       0.200418            0.267201   
35  ACB  2016  2                 15.567581       0.188499            0.260659   
36  ACB  2016  1                 15.016311       0.189328            0.255096   
37  ACB  2015  4                  0.000000       0.193905            0.248947   
38  ACB  2015  3                  0.000000       0.196589            0.245588   
39  ACB  2015  2                 14.353217       0.206608            0.240282   
40  ACB  2015  1                  0.000000       0.201880            0.246851   
41  ACB  2014  4                 13.487810       0.226223            0.241350   
42  ACB  2014  3                 13.342019       0.211001            0.243493   
43  ACB  2014  2                 13.446861       0.217349            0.238914   
44  ACB  2014  1                 12.586751       0.196432            0.248444   
45  ACB  2013  4                 12.323440       0.204186            0.243391   
46  ACB  2013  3                 11.526566       0.133516            0.249403   
47  ACB  2013  2                 1

In [22]:
stock = Vnstock().stock(symbol='ACB')
df = stock.quote.history(start=start_date, end=end_date)
finance_data = stock.finance.ratio(period='quarter', lang='vi', dropna=True)

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


In [23]:
finance_data

Meta          Chỉ tiêu cơ cấu nguồn vốn                                     \
     CP   Năm Kỳ                   Nợ/VCSH TSCĐ / Vốn CSH Vốn CSH/Vốn điều lệ   
0   ACB  2025  1                  9.240355       0.061005            1.695161   
1   ACB  2024  4                  9.352125       0.064849            1.624825   
2   ACB  2024  3                  8.858604       0.058881            1.535128   
3   ACB  2024  2                  9.290830       0.062380            1.456058   
4   ACB  2024  1                  8.724440       0.062450            1.456020   
5   ACB  2023  4                  9.130151       0.067120            1.381365   
6   ACB  2023  3                  8.704559       0.066554            1.300949   
7   ACB  2023  2                  9.035334       0.068278            1.223893   
8   ACB  2023  1                  8.774859       0.066006            1.217332   
9   ACB  2022  4                  9.401935       0.068129            1.137679   
10  ACB  2022  3                  9.067578       0.066993            1.085039   
11  ACB  2022  2                  9.443214       0.069625            1.013617   
12  ACB  2022  1                  9.966802       0.076310            0.938418   
13  ACB  2021  4                 10.754104       0.083797            0.874127   
14  ACB  2021  3                 10.282250       0.085829            0.827064   
15  ACB  2021  2                 10.670941       0.091710            0.786119   
16  ACB  2021  1                 10.851374       0.098095            0.738405   
17  ACB  2020  4                 11.540286       0.106712            0.690102   
18  ACB  2020  3                 11.720578       0.113291            0.640864   
19  ACB  2020  2                 11.868276       0.121825            0.600243   
20  ACB  2020  1                 12.171197       0.128266            0.572597   
21  ACB  2019  4                 12.812695       0.135851            0.540283   
22  ACB  2019  3                 13.121225       0.137624            0.493790   
23  ACB  2019  2                 13.724449       0.139122            0.463993   
24  ACB  2019  1                 14.002276       0.143898            0.435759   
25  ACB  2018  4                 14.669180       0.153840            0.409175   
26  ACB  2018  3                 15.024454       0.160896            0.379990   
27  ACB  2018  2                  0.000000       0.170651            0.356630   
28  ACB  2018  1                 16.456520       0.176367            0.334405   
29  ACB  2017  4                 16.735565       0.187614            0.312087   
30  ACB  2017  3                 16.523023       0.196855            0.300261   
31  ACB  2017  2                 16.860526       0.204633            0.289767   
32  ACB  2017  1                 16.280560       0.199433            0.283376   
33  ACB  2016  4                 15.617052       0.202703            0.273772   
34  ACB  2016  3                 15.674916       0.200418            0.267201   
35  ACB  2016  2                 15.567581       0.188499            0.260659   
36  ACB  2016  1                 15.016311       0.189328            0.255096   
37  ACB  2015  4                  0.000000       0.193905            0.248947   
38  ACB  2015  3                  0.000000       0.196589            0.245588   
39  ACB  2015  2                 14.353217       0.206608            0.240282   
40  ACB  2015  1                  0.000000       0.201880            0.246851   
41  ACB  2014  4                 13.487810       0.226223            0.241350   
42  ACB  2014  3                 13.342019       0.211001            0.243493   
43  ACB  2014  2                 13.446861       0.217349            0.238914   
44  ACB  2014  1                 12.586751       0.196432            0.248444   
45  ACB  2013  4                 12.323440       0.204186            0.243391   
46  ACB  2013  3                 11.526566       0.133516            0.249403   
47  ACB  2013  2                 1

In [18]:
def crawl_stock_data_FA_TA(symbols, start_date, end_date, save_folder='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/test_data'):
    """
    Crawl dữ liệu lịch sử giá cổ phiếu cho danh sách mã, lưu mỗi mã vào 1 file Excel riêng.

    Parameters:
        symbols (list): Danh sách mã cổ phiếu (ví dụ: ['VCB', 'FPT', 'MWG'])
        start_date (str): Ngày bắt đầu (ví dụ: '2020-01-01')
        end_date (str): Ngày kết thúc (ví dụ: '2025-01-01')
        save_folder (str): Tên thư mục để lưu file Excel (mặc định là 'stock_data')
    """

    # Tạo thư mục lưu file nếu chưa tồn tại
    os.makedirs(save_folder, exist_ok=True)
    indicators = {
    'P/B': ('Chỉ tiêu định giá', 'P/B'),
    'P/E': ('Chỉ tiêu định giá', 'P/E'),
    'P/S': ('Chỉ tiêu định giá', 'P/S'),
    'P/Cash Flow': ('Chỉ tiêu định giá', 'P/Cash Flow'),
    'EPS (VND)': ('Chỉ tiêu định giá', 'EPS (VND)'),
    'BVPS (VND)': ('Chỉ tiêu định giá', 'BVPS (VND)'),
    'EV/EBITDA': ('Chỉ tiêu định giá', 'EV/EBITDA'),
    'ROE (%)': ('Chỉ tiêu khả năng sinh lợi', 'ROE (%)'),
    'ROIC (%)': ('Chỉ tiêu khả năng sinh lợi', 'ROIC (%)'),
    'ROA (%)': ('Chỉ tiêu khả năng sinh lợi', 'ROA (%)'),
}

    for symbol in symbols:
        try:
            print(f"Đang lấy dữ liệu cho: {symbol}...")
            stock = Vnstock().stock(symbol=symbol)
            df = stock.quote.history(start=start_date, end=end_date)
            finance_data = stock.finance.ratio(period='quarter', lang='vi', dropna=True)

            for index, row in df.iterrows():
                input_date = row['time']
                prev_quarter, prev_year, year_before = get_previous_and_last_year_quarter(input_date)

                for name, multi_idx in indicators.items():
                    col_prev = f"{name}_Previous_Quarter"
                    col_last_year = f"{name}_Same_Period_Last_Year"

                    # Giá trị quý trước
                    try:
                        value_prev = finance_data.loc[
                            (finance_data[('Meta', 'Năm')].astype(str) == str(prev_year)) &
                            (finance_data[('Meta', 'Kỳ')].astype(str) == str(prev_quarter)),
                            multi_idx
                        ].values
                        df.at[index, col_prev] = value_prev[0] if len(value_prev) > 0 else None
                    except:
                        df.at[index, col_prev] = None

                    # Giá trị cùng kỳ năm trước
                    try:
                        value_last_year = finance_data.loc[
                            (finance_data[('Meta', 'Năm')].astype(str) == str(year_before)) &
                            (finance_data[('Meta', 'Kỳ')].astype(str) == str(prev_quarter)),
                            multi_idx
                        ].values
                        df.at[index, col_last_year] = value_last_year[0] if len(value_last_year) > 0 else None
                    except:
                        df.at[index, col_last_year] = None

                    # Thêm 8 quý gần nhất
                    recent_quarters = get_last_n_quarters(input_date, n=8)
                    for i, (q, y) in enumerate(recent_quarters):
                        col_recent = f"{name}_d_{i+1}"
                        try:
                            value = finance_data.loc[
                                (finance_data[('Meta', 'Năm')].astype(str) == str(y)) &
                                (finance_data[('Meta', 'Kỳ')].astype(str) == str(q)),
                                multi_idx
                            ].values
                            df.at[index, col_recent] = value[0] if len(value) > 0 else 0
                        except:
                            df.at[index, col_recent] = 0
                    # Tính khoảng cách từ input_date đến cuối quý gần nhất
                    try:
                        end_of_last_quarter = get_latest_quarter_end_date(input_date)
                        distance = (pd.to_datetime(input_date) - end_of_last_quarter).days
                        df.at[index, 'distance_to_nearest_quarter'] = distance
                    except:
                        df.at[index, 'distance_to_nearest_quarter'] = None

            df['ticket'] = symbol  # 👉 Thêm cột 'ticket'
            file_path = os.path.join(save_folder, f"{symbol}.xlsx")
            df.to_excel(file_path, index=False)
            print(f"✅ Đã lưu dữ liệu {symbol} vào {file_path}")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý mã {symbol}: {e}")

In [ ]:
def crawl_stock_data_TA(symbols, start_date, end_date, save_folder='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA/v1/test_data'):
    """
    Crawl dữ liệu lịch sử giá cổ phiếu cho danh sách mã, lưu mỗi mã vào 1 file Excel riêng.

    Parameters:
        symbols (list): Danh sách mã cổ phiếu (ví dụ: ['VCB', 'FPT', 'MWG'])
        start_date (str): Ngày bắt đầu (ví dụ: '2020-01-01')
        end_date (str): Ngày kết thúc (ví dụ: '2025-01-01')
        save_folder (str): Tên thư mục để lưu file Excel (mặc định là 'stock_data')
    """

    # Tạo thư mục lưu file nếu chưa tồn tại
    os.makedirs(save_folder, exist_ok=True)

    for symbol in symbols:
        try:
            print(f"Đang lấy dữ liệu cho: {symbol}...")
            stock = Vnstock().stock(symbol=symbol)
            df = stock.quote.history(start=start_date, end=end_date)
            # finance_data = stock.finance.ratio(period='quarter', lang='vi', dropna=True)


            df['ticket'] = symbol  # 👉 Thêm cột 'ticket'
            file_path = os.path.join(save_folder, f"{symbol}.xlsx")
            df.to_excel(file_path, index=False)
            print(f"✅ Đã lưu dữ liệu {symbol} vào {file_path}")

        except Exception as e:
            print(f"❌ Lỗi khi xử lý mã {symbol}: {e}")

In [20]:
# crawl train data
crawl_stock_data_FA_TA(['ACB'], start_date='2018-01-01', end_date='2023-12-31', save_folder=r'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\TA_FA\data\test')

Đang lấy dữ liệu cho: ACB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

✅ Đã lưu dữ liệu ACB vào D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\TA_FA\data\test\ACB.xlsx


In [ ]:
# crawl train data
crawl_stock_data_FA_TA(['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE'], start_date='2018-01-01', end_date='2023-12-31', save_folder='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2')

Đang lấy dữ liệu cho: ACB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

✅ Đã lưu dữ liệu ACB vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2/ACB.xlsx
Đang lấy dữ liệu cho: BCM...


DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

✅ Đã lưu dữ liệu BCM vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2/BCM.xlsx
Đang lấy dữ liệu cho: BID...


...

✅ Đã lưu dữ liệu VNM vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2/VNM.xlsx
Đang lấy dữ liệu cho: VPB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newfr

✅ Đã lưu dữ liệu VPB vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2/VPB.xlsx
Đang lấy dữ liệu cho: VRE...


DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, us

✅ Đã lưu dữ liệu VRE vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2/VRE.xlsx


In [ ]:
crawl_stock_data_TA(['VNINDEX'], start_date='2018-01-01', end_date='2023-12-31', save_folder='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2')

2025-04-23 16:50:12 - vnstock.common.data.data_explorer - INFO - Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.
INFO:vnstock.common.data.data_explorer:Không phải là mã chứng khoán, thông tin công ty và tài chính không khả dụng.


Đang lấy dữ liệu cho: VNINDEX...
✅ Đã lưu dữ liệu VNINDEX vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train_v2/VNINDEX.xlsx


In [ ]:
# crawl test data
crawl_stock_data_FA_TA(['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE'], start_date='2024-01-01', end_date='2024-12-31', save_folder='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data')

Đang lấy dữ liệu cho: ACB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Đã lưu dữ liệu ACB vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/ACB.xlsx
Đang lấy dữ liệu cho: BCM...
✅ Đã lưu dữ liệu BCM vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/BCM.xlsx
Đang lấy dữ liệu cho: BID...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Đã lưu dữ liệu BID vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/BID.xlsx
Đang lấy dữ liệu cho: BVH...


...

✅ Đã lưu dữ liệu VCB vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VCB.xlsx
Đang lấy dữ liệu cho: VHM...
✅ Đã lưu dữ liệu VHM vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VHM.xlsx
Đang lấy dữ liệu cho: VIB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Đã lưu dữ liệu VIB vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VIB.xlsx
Đang lấy dữ liệu cho: VIC...
✅ Đã lưu dữ liệu VIC vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VIC.xlsx
Đang lấy dữ liệu cho: VJC...
✅ Đã lưu dữ liệu VJC vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VJC.xlsx
Đang lấy dữ liệu cho: VNM...
✅ Đã lưu dữ liệu VNM vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VNM.xlsx
Đang lấy dữ liệu cho: VPB...


Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


✅ Đã lưu dữ liệu VPB vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VPB.xlsx
Đang lấy dữ liệu cho: VRE...
✅ Đã lưu dữ liệu VRE vào /content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v1/test_data/VRE.xlsx
